### SQLAlchemy

SQLAlchemy
- http://docs.sqlalchemy.org/en/latest/orm/tutorial.html
- `pip3 install sqlalchemy`
- python에서 사용하는 ORM

ORM이란?
- Object-relational mapping
- 데이터 베이스를 객체화 시켜 데이터베이스에 있는 데이터를 CRUD를 할수 있음.
- 사용하는 DB를 변경해야 할 경우 데이터 베이스 엔진만 바꾸면 됌

1. db connect
2. mapping class (db.table - class)
3. insert
4. select
5. update
6. delete

In [9]:
import sqlalchemy, pickle
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker

##### db connect

In [10]:
# mysql+mysqldb://<사용자계정>:<비밀번호>@<호스트>/<데이터베이스_이름>
pw = '123456Cc'
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@13.125.105.201/test")

##### mapping class

In [11]:
# __tablename__ : 데이터베이스의 테이블 이름

Base = declarative_base()
class User(Base):
    __tablename__ = 'user2'

    user_id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    age = Column(Integer)
    rdate = Column(DateTime)
    
    def __init__(self, name, email, age, rdate):
        self.name = name
        self.email = email
        self.age = age
        self.rdate = rdate
    
    def __repr__(self):
        return "<User {}, {}, {}, {}>".format(self.name, self.email, self.age, self.rdate)

In [12]:
# engine에 연결된 데이터 베이스 테이블 생성
Base.metadata.create_all(engine)

OperationalError: (_mysql_exceptions.OperationalError) (1045, "Access denied for user 'root'@'121.130.43.4' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [13]:
# make session
Session = sessionmaker(bind=engine)
session = Session()

##### insert

In [14]:
# add user object to session
# add one object
user = User('jin', 'jin@gmail.com', '27', '2016-03-21')
session.add(user)

In [15]:
# add multy objects
users = [
    User('alice', 'alice@gmail.com', '25', '2018-02-21'),
    User('andy', 'andy@daum.net', '33', '2015-01-19'),
]
session.add_all(users)

In [16]:
# run transaction
session.commit()

OperationalError: (_mysql_exceptions.OperationalError) (1045, "Access denied for user 'root'@'121.130.43.4' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [9]:
# rollback (session data clear)
session.rollback()

##### select
- filter
- order by

In [10]:
def disp(datas):
    for data in datas:
        print(data)

In [11]:
results = session.query(User).all()
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [13]:
# filter
# ==, !=, >, <, >=, <=, like, in_, ~, 
results = session.query(User).filter(User.name == "jin")
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [14]:
results = session.query(User).filter(User.name != "jin")
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [15]:
results = session.query(User).filter(User.age > 26)
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [16]:
results = session.query(User).filter(User.email.like("%gmail%"))
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [18]:
results = session.query(User).filter(User.name.in_(["alice","andy"]))
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [19]:
results = session.query(User).filter(~User.email.like("%gmail%"))
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [21]:
# and_, or_
from sqlalchemy import and_, or_
results = session.query(User).filter(or_(User.name == "jin", User.age == 33))
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [22]:
results = session.query(User).filter(and_(User.age < 30, User.email.like("%gmail%")))
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [29]:
# order_by
results = session.query(User).order_by(User.age.asc())
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [30]:
results = session.query(User).order_by(User.age)
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [31]:
results = session.query(User).order_by(User.age.desc())
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [33]:
# count
session.query(User).count()

3

##### update

In [39]:
jin = session.query(User).filter(User.name == "jin").one()

In [41]:
jin.age = 30

In [42]:
session.add(jin)

In [43]:
session.commit()

##### delete

In [44]:
session.query(User).filter(User.name == "jin").delete()

1

In [45]:
session.commit()

In [46]:
# close session
session.close()